<a href="https://colab.research.google.com/github/Artmagic/GP2_team34/blob/dev/%D0%9F%D0%B0%D1%81%D1%81%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%BF%D0%BE%D1%82%D0%BE%D0%BA_%D0%BF%D0%BE_%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D1%8F%D0%BC_%D0%BC%D0%B5%D1%82%D1%80%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

url = "https://apidata.mos.ru/v1/datasets"

response = requests.get(url, params={"$top": 2000, "api_key": "a062e7e5-7df3-470f-8751-1b48ffce3931"})
data = response.json()

items = pd.DataFrame(data["Items"])
items.head()

,CategoryId,PublishDate,Keywords,IdentificationNumber,ContainsGeodata,IsSeasonal,IsNew,VersionNumber,Caption,ContainsAccEnvData,IsForeign,IsArchive,SefUrl,Id,ReleaseNumber,DepartmentId,Season,FullDescription,LastUpdateDate
0,107,2014-05-13,"библиотеки, выставка, выставочные залы, дворец...",7702155262-culturehouse,True,False,False,3,Дома культуры и клубы,True,False,False,None,493,1094,1008,0,<p>Набор данных содержит информацию о домах ку...,2025-10-30
1,111,2014-04-10,"библиотеки, выставочные залы, дома культуры и ...",7702155262-MovieTheaters,True,False,False,3,Кинотеатры,True,False,False,None,495,58,1008,0,"<p style=""margin-left:0cm; margin-right:0cm"">Н...",2025-10-30
2,111,2014-01-15,"аттракцион, аттракционы",7709083955-attractions,True,False,False,7,Аттракционы в парках и на площадках,False,False,False,None,498,555,1214,0,<!--[if gte mso 9]><xml>\n <o:OfficeDocumentSe...,2025-11-06
3,112,2014-04-03,"государственные учреждения, государственное уч...",7701236617-DZKHIBgos,True,False,False,3,"Организации, находящиеся в ведомственном подчи...",False,False,False,None,499,26,1006,0,"<p><span style=""font-family:Tahoma; font-size:...",2024-12-06
4,112,2014-04-03,"инженерные службы, инженерная служба, службы, ...",7701236617-Regionalengineeringservices,True,False,False,3,Государственные казенные учреждения «Инженерны...,True,False,True,None,500,30,1006,0,"<p style=""margin-bottom:0cm;margin-bottom:.000...",2022-01-21


In [2]:
items[items["Caption"].str.contains("поток", case=False, na=False)][["Id", "Caption"]]

,Id,Caption
1116,62521,Месячный пассажиропоток по всем видам обществе...
1117,62523,Годовой пассажиропоток по всем видам обществен...
1129,62743,Пассажиропоток по станциям Московского метропо...


In [7]:
import pandas as pd
import time

base_url = f"https://apidata.mos.ru/v1/datasets/62743/rows"
page_size = 1000
skip = 0
all_records = []

while True:
    params = {
        "api_key": "a062e7e5-7df3-470f-8751-1b48ffce3931",
        "$top": page_size,
        "$skip": skip
    }
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Ошибка {response.status_code} при skip={skip}")
        break

    batch = response.json()
    if not batch:
        break

    # Извлекаем "Cells"
    all_records.extend([b["Cells"] for b in batch])

    skip += page_size

    # Небольшая пауза, чтобы не перегружать API
    time.sleep(0.5)

df = pd.DataFrame(all_records)
print("Итоговый размер:", df.shape)


Итоговый размер: (5533, 7)


In [9]:
df

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers,global_id
0,Новаторская,Троицкая линия,2025,III квартал,1167230,1181704,2822445543
1,Университет Дружбы Народов,Троицкая линия,2025,III квартал,887396,877567,2822436510
2,Генерала Тюленева,Троицкая линия,2025,III квартал,473206,474053,2822436511
3,Тютчевская,Троицкая линия,2025,III квартал,310532,314382,2822436512
4,Коммунарка,Троицкая линия,2025,III квартал,517226,523637,2822436515
...,...,...,...,...,...,...,...
5528,Деловой центр,Большая кольцевая линия,2021,I квартал,1062070,1053347,1139050976
5529,Хорошёвская,Большая кольцевая линия,2021,I квартал,687219,687558,1139049567
5530,Петровский парк,Большая кольцевая линия,2021,I квартал,752848,778402,1138976383
5531,ЦСКА,Большая кольцевая линия,2021,I квартал,2337493,2374510,1138976435


In [18]:
station_year_passenger_flow = df.groupby(['NameOfStation', 'Line', 'Year'])[['IncomingPassengers', 'OutgoingPassengers']].sum().reset_index()
station_year_passenger_flow

,NameOfStation,Line,Year,IncomingPassengers,OutgoingPassengers
0,Авиамоторная,Большая кольцевая линия,2023,4081013,4213733
1,Авиамоторная,Большая кольцевая линия,2024,4384158,4845349
2,Авиамоторная,Большая кольцевая линия,2025,3854426,4165770
3,Авиамоторная,Калининская линия,2021,12933490,13555698
4,Авиамоторная,Калининская линия,2022,14050604,14720309
...,...,...,...,...,...
1483,улица Дмитриевского,Некрасовская линия,2021,2896953,2819403
1484,улица Дмитриевского,Некрасовская линия,2022,3142157,3055071
1485,улица Дмитриевского,Некрасовская линия,2023,3390407,3382136
1486,улица Дмитриевского,Некрасовская линия,2024,3421243,3421946


In [19]:
station_year_passenger_flow = station_year_passenger_flow[station_year_passenger_flow.Year == 2024]
station_year_passenger_flow

,NameOfStation,Line,Year,IncomingPassengers,OutgoingPassengers
1,Авиамоторная,Большая кольцевая линия,2024,4384158,4845349
6,Авиамоторная,Калининская линия,2024,11425579,11961284
13,Автозаводская,Замоскворецкая линия,2024,12952791,13584866
18,Автозаводская,Московское центральное кольцо,2024,1849625,2039005
23,Академическая,Калужско-Рижская линия,2024,11595003,11683305
...,...,...,...,...,...
1468,Юго-Западная,Сокольническая линия,2024,18541254,18819195
1473,Южная,Серпуховско-Тимирязевская линия,2024,8034606,7968476
1478,Ясенево,Калужско-Рижская линия,2024,10696850,10550298
1481,Яхромская,Люблинско-Дмитровская линия,2024,6312744,6162648


In [20]:
del station_year_passenger_flow['Line'], station_year_passenger_flow['Year']
station_year_passenger_flow

,NameOfStation,IncomingPassengers,OutgoingPassengers
1,Авиамоторная,4384158,4845349
6,Авиамоторная,11425579,11961284
13,Автозаводская,12952791,13584866
18,Автозаводская,1849625,2039005
23,Академическая,11595003,11683305
...,...,...,...
1468,Юго-Западная,18541254,18819195
1473,Южная,8034606,7968476
1478,Ясенево,10696850,10550298
1481,Яхромская,6312744,6162648


In [21]:
station_total_passenger_flow = station_year_passenger_flow.groupby('NameOfStation')[['IncomingPassengers', 'OutgoingPassengers']].sum().reset_index()
station_total_passenger_flow

,NameOfStation,IncomingPassengers,OutgoingPassengers
0,Авиамоторная,15809737,16806633
1,Автозаводская,14802416,15623871
2,Академическая,11595003,11683305
3,Александровский сад,6704109,6790020
4,Алексеевская,11082282,11310666
...,...,...,...
258,Юго-Западная,18541254,18819195
259,Южная,8034606,7968476
260,Ясенево,10696850,10550298
261,Яхромская,6312744,6162648


In [22]:
station_total_passenger_flow.to_csv('station_total_passenger_flow.csv', index=False)